In [ ]:
from pamod.preprocessing import StaticProcessEngine
from pamod.descriptives import DescriptivesPlotter

In [ ]:
engine = StaticProcessEngine(behavior=False, scale=False, encoding=None)
df = engine.load_data()
df = engine.process_data(df)
plotter = DescriptivesPlotter(df)

In [ ]:
plotter.histogram_2d("pdbaseline", "pdrevaluation")

In [ ]:
plotter.pocket_comparison("pdbaseline", "pdrevaluation")

In [ ]:
plotter.pocket_group_comparison("pdbase", "pdgroup")

In [ ]:
plotter.plt_matrix("pdgroup", "pdbase", "Blues", len(df["pdgroup"]), "columns")